In [2]:
from transformers import pipeline
import pprint

pp = pprint.PrettyPrinter(indent=0,compact=True)
import pandas as pd

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive"

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd

data = pd.read_csv("/content/drive/My Drive/labeled_data.csv")

In [ ]:
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
data["class"].unique()

array([2, 1, 0])

In [ ]:
toxic_data = list(data[data["class"] == 1]["tweet"])

In [ ]:
def clean_data(tweet):
  idx = tweet.find(":")

  return tweet[idx+1:].strip()

In [ ]:
cleaned_data = pd.Series(toxic_data).map(clean_data)

In [ ]:
import re

In [ ]:
import re

url_pattern = r'https?://\S+|t\.\S+'

def remove_gibberish(text):
    text = re.sub(url_pattern, '', text)
    text = re.sub(r'&#\d+;','', text)
    text = re.sub(r'\s+'," ", text)
    return text.strip()

cleaned_data = [remove_gibberish(sentence) for sentence in cleaned_data]


In [ ]:
import re

def clean_text(text):
    # Keeping alphanumeric characters, spaces, and some basic punctuation
    pattern = re.compile(r'[^\w\s,.!?\']')
    text = pattern.sub(' ', text)
    text = text.strip('\'\"')
    return text.strip()

data_prep_cleaned = []
for entry in data_prep:
    cleaned_context = clean_text(entry["context"])
    data_prep_cleaned.append({
        "context": cleaned_context,
        "question": entry["question"],
        "answers": entry["answers"]
})

In [ ]:
# data_prep_cleaned
import json

with open("cleaned_data.json", "w") as f:
  json.dump(data_prep_cleaned, f)

In [ ]:
def get_sentence(li):

  filtered_comments = []

  for i in li:
    if len(i) <= 100:
      filtered_comments.append(i)

  return filtered_comments

In [ ]:
df = pd.DataFrame(data_prep)

In [ ]:
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [ ]:
import json

with open("/content/cleaned_data.json") as f:
  new_data = json.load(f)

In [ ]:
new_data[:2]

[{'context': 'boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!',
  'question': 'What all toxic words are used in this sentence?',
  'answers': [{'text': 'hoe', 'start_position': 44}]},
 {'context': 'You ever fuck a bitch and she start to cry? You be confused as shit',
  'question': 'What all toxic words are used in this sentence?',
  'answers': [{'text': 'fuck a bitch', 'start_position': 9},
   {'text': 'shit', 'start_position': 63}]}]

In [ ]:
def correct_start_positions(data):
    for entry in data:
        context = entry["context"]
        new_answers = []
        processed = set()
        for answer in entry["answers"]:
            text = answer["text"]
            start = 0
            while start < len(context):
                position = context.find(text, start)
                if position == -1:
                    break
                # Check if this occurrence has already been processed
                if (text, position) not in processed:
                    new_answers.append({'text': text, 'start_position': position})
                    processed.add((text, position))
                start = position + 1
        entry["answers"] = new_answers
    return data


In [ ]:
updated_data = correct_start_positions(new_data)

In [ ]:
corrected_data = []

for query in updated_data:
  d = { "question" : query["question"],"context" : query["context"], "answers" : {'answer_start': [],'text':[] }}

  for _ in query["answers"]:
    d["answers"]["answer_start"].append(_["start_position"])
    d["answers"]["text"].append(_["text"])

  corrected_data.append(d)


In [ ]:
corrected_data

In [ ]:
data = pd.DataFrame(corrected_data)

In [ ]:
data

,question,context,answers
0,What all toxic words are used in this sentence?,boy dats cold...tyga dwn bad for cuffin dat ho...,"{'answer_start': [44], 'text': ['hoe']}"
1,What all toxic words are used in this sentence?,You ever fuck a bitch and she start to cry? Yo...,"{'answer_start': [9, 63], 'text': ['fuck a bit..."
2,What all toxic words are used in this sentence?,viva_based she look like a tranny,"{'answer_start': [27, 11], 'text': ['tranny', ..."
3,What all toxic words are used in this sentence?,The shit you hear about me might be true or it...,"{'answer_start': [4, 71, 77], 'text': ['shit y..."
4,What all toxic words are used in this sentence?,The shit just blows me..claim you so faithful ...,"{'answer_start': [4, 78, 91], 'text': ['shit j..."
...,...,...,...
194,What all toxic words are used in this sentence?,Dumb Niggaz. Yall Niggas are dumb. Sean Kingst...,"{'answer_start': [0, 18, 90], 'text': ['Dumb N..."
195,What all toxic words are used in this sentence?,Hey I'm just tellin' like it is you ugly as fu...,"{'answer_start': [36, 49], 'text': ['ugly as f..."
196,What all toxic words are used in this sentence?,You are a raging faggot. Kill yourself.,"{'answer_start': [10, 26], 'text': ['raging fa..."
197,What all toxic words are used in this sentence?,cocksucking bastard is gonna go hill,"{'answer_start': [0], 'text': ['cocksucking ba..."


In [ ]:
import json

# data.to_csv('train_data.csv', index=False)

with open("train_data.json", "w") as f:
  json.dump(corrected_data,f)

In [ ]:
import json

with open("/content/new_train_data.json", "r") as f:
  data = json.load(f)

In [ ]:
data[1]

{'question': 'What toxic sentence is used in this paragraph?',
 'context': "You ever fuck a bitch and she start to cry? You be confused as shit. I got this for my Mum who is not diabetic but needs to watch her sugar intake, and my father who simply chooses to limit unnecessary sugar intake - she's the one with the sweet tooth - they both LOVED these toffees, you would never guess that they're sugar-free and it's so great that you can eat them pretty much guilt free! i was so impressed that i've ordered some for myself (w dark chocolate) to take to the office so i'll eat them instead of snacking on sugary sweets. These are just EXCELLENT!",
 'answers': {'answer_start': [0],
  'text': ['You ever fuck a bitch and she start to cry? You be confused as shit']}}

In [ ]:
from datasets import load_dataset

In [ ]:
raw_data = load_dataset('json', data_files='/content/new_train_data.json')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
raw_data

DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'question'],
        num_rows: 199
    })
})

In [ ]:
raw_data["train"][0]

{'answers': {'answer_start': [0],
  'text': ['boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!']},
 'context': 'boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!. Product received is as advertised. <a href=" Strawberry, 16-Ounce Bags (Pack of 6)</a>',
 'question': 'What toxic sentence is used in this paragraph?'}

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "rsvp-ai/bertserini-bert-base-squad"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
pp.pprint(tokenizer(raw_data["train"][0]["question"],raw_data["train"][0]["context"]))

inputs = tokenizer(raw_data["train"][0]["question"],raw_data["train"][0]["context"])

tokenizer.decode(inputs["input_ids"])

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                  1],
'input_ids': [101, 2054, 11704, 6251, 2003, 2109, 1999, 2023, 20423, 1029, 102,
             2879, 23755, 2015, 3147, 1012, 1012, 1012, 5939, 3654, 1040, 7962,
             2919, 2005, 26450, 2378, 23755, 7570, 2063, 1999, 1996, 3083, 2173,
             999, 999, 1012, 4031, 2363, 2003, 2004, 17099, 1012, 1026, 1037,
             17850, 12879, 1027, 1000, 16876, 1010, 2385, 1011, 19471, 8641,
             1006, 5308, 1997, 1020, 1007, 1026, 1013, 1037, 1028, 102],
'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                  1]

'[CLS] what toxic sentence is used in this paragraph? [SEP] boy dats cold... tyga dwn bad for cuffin dat hoe in the 1st place!!. product received is as advertised. < a href = " strawberry, 16 - ounce bags ( pack of 6 ) < / a > [SEP]'

In [ ]:
inputs = tokenizer(
    raw_data["train"][66]["question"],
    raw_data["train"][66]["context"],
    max_length = 500,
    truncation="only_second",
    stride = 50,
    return_overflowing_tokens= True,
    return_offsets_mapping = True
)

In [ ]:
pp.pprint(inputs)

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                  1, 1, 1, 1]],
'input_ids': [[101, 2054, 2035, 11704, 2616, 2024, 2109, 1999, 2023, 6251, 1029,
             102, 1057, 2412, 3102, 1037, 14405, 2006, 1996, 11996, 1998, 2228,
             4365, 2054, 2065, 2008, 9152, 23033, 2001, 2006, 2010, 2126, 2000,
             2131, 2070, 22418, 1005, 1005, 2053, 18667, 2442, 1038, 14181,
             2382, 3382, 102]],
'offset_mapping': [[(0, 0), (0, 4), (5, 8), (9, 14), (15, 20), (21, 24), (25, 29),
                  (30, 32), (33, 37), (38, 46), (46, 47), (0, 0), (0, 1),
                  (2, 6), (7, 11), (12, 13), (14, 17), (18, 20), (21, 24),
                  (25, 33), (34, 37), (38, 43), (44, 48), (49, 53), (54, 56),
                  (57, 61), (62, 64), (64, 67), (68, 71), (72, 74), (75, 78),
                  (79, 82), (83, 85), (86, 89), (90, 94), (95, 100

In [ ]:
inputs.sequence_ids

<bound method BatchEncoding.sequence_ids of {'input_ids': [[101, 2054, 2035, 11704, 2616, 2024, 2109, 1999, 2023, 6251, 1029, 102, 1057, 2412, 3102, 1037, 14405, 2006, 1996, 11996, 1998, 2228, 4365, 2054, 2065, 2008, 9152, 23033, 2001, 2006, 2010, 2126, 2000, 2131, 2070, 22418, 1005, 1005, 2053, 18667, 2442, 1038, 14181, 2382, 3382, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'offset_mapping': [[(0, 0), (0, 4), (5, 8), (9, 14), (15, 20), (21, 24), (25, 29), (30, 32), (33, 37), (38, 46), (46, 47), (0, 0), (0, 1), (2, 6), (7, 11), (12, 13), (14, 17), (18, 20), (21, 24), (25, 33), (34, 37), (38, 43), (44, 48), (49, 53), (54, 56), (57, 61), (62, 64), (64, 67), (68, 71), (72, 74), (75, 78), (79, 82), (83, 85), (8

In [ ]:
pp.pprint(inputs.sequence_ids(0))

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]


In [ ]:
pp.pprint(inputs["offset_mapping"])

[[(0, 0), (0, 4), (5, 8), (9, 14), (15, 20), (21, 24), (25, 29), (30, 32),
(33, 37), (38, 46), (46, 47), (0, 0), (0, 1), (2, 6), (7, 11), (12, 13),
(14, 17), (18, 20), (21, 24), (25, 33), (34, 37), (38, 43), (44, 48), (49, 53),
(54, 56), (57, 61), (62, 64), (64, 67), (68, 71), (72, 74), (75, 78), (79, 82),
(83, 85), (86, 89), (90, 94), (95, 100), (100, 101), (102, 103), (103, 105),
(106, 108), (109, 113), (114, 115), (117, 119), (120, 122), (123, 129),
(0, 0)]]


In [ ]:
!pip install datasets

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

raw_data = load_dataset('json', data_files='/content/new_train_data.json')

model_checkpoint = "rsvp-ai/bertserini-bert-base-squad"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Load your dataset
raw_data = load_dataset('json', data_files='/content/new_train_data.json')

# Load the tokenizer
model_checkpoint = "rsvp-ai/bertserini-bert-base-squad"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Function to preprocess the data
def prepare_train_features(examples):
    # Tokenize the questions and contexts together as a pair
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",  # Truncate only the context part
        max_length=512,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    # Let's label those examples!
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)

        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]

        # If no answers are provided, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

# Apply the function to preprocess the data
tokenized_dataset = raw_data.map(prepare_train_features, batched=True)

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

In [ ]:
pp.pprint(tokenized_dataset["train"][0])

{'answers': {'answer_start': [0],
           'text': ['boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st '
                   'place!!']},
'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                  1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

def train_model(epochs):
  model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

  training_args = TrainingArguments(
      "toxic_analyzer",
      evaluation_strategy="epoch",
      learning_rate=2e-5,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      num_train_epochs=epochs,
      weight_decay=0.01,
      fp16=True,
  )

  trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"].remove_columns(['question', 'context', 'answers']),
    eval_dataset=tokenized_dataset["train"].remove_columns(['question', 'context', 'answers']),
    tokenizer=tokenizer
  )

  trainer.train()

  return trainer

In [ ]:
trainer_1 = train_model(1)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at rsvp-ai/bertserini-bert-base-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.724888


In [ ]:
trainer_2 = train_model(2)

Some weights of the model checkpoint at rsvp-ai/bertserini-bert-base-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch,Training Loss,Validation Loss
1,No log,0.571223
2,No log,0.224260


In [ ]:
trainer_3 = train_model(3)

Some weights of the model checkpoint at rsvp-ai/bertserini-bert-base-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch,Training Loss,Validation Loss
1,No log,0.529068
2,No log,0.118603
3,No log,0.061158


In [ ]:
trainer_1.save_model("./trained_model_ep_1")
trainer_2.save_model("./trained_model_ep_2")
trainer_3.save_model("./trained_model_ep_3")

In [2]:
from transformers import pipeline

qa = pipeline(
  "question-answering",
  model='/content/drive/MyDrive/trained_model_ep_1'
)

qa2 = pipeline(
  "question-answering",
  model='/content/drive/MyDrive/trained_model_ep_2'
)

qa3 = pipeline(
  "question-answering",
  model='/content/drive/MyDrive/trained_model_ep_3'
)

In [6]:
def get_toxic_statement(context):
    question = "What toxic sentence is used in this paragraph?"
    votes = {}

    m1 = qa(context=context, question=question)

    votes[m1["answer"]] = 1

    m2 = qa2(context=context, question=question)

    if votes.get(m2["answer"]):
         votes[m2["answer"]] =+ 1
    else:
        votes[m2["answer"]] = 1

    m3 = qa3(context=context, question=question)

    if votes.get(m3["answer"]):
         votes[m3["answer"]] =+ 1
    else:
        votes[m3["answer"]] = 1

    if len(votes.keys()) == 1:
        return votes.keys()[0]
    elif m1["score"] > m2["score"]:
        return m1["answer"]
    elif m2["score"] > m3["score"]:
        return m2["answer"]
    else:
        return m3["answer"]


In [7]:
get_toxic_statement("Hey, how you doing. Its wonderfull day oputside right? I wanna c*cksucker you!!")

'I wanna c*cksucker you!!'

In [3]:
# context = "Hey, how you doing. Its wonderfull day oputside right? I wanna c*cksucker you!!"
# question = "What toxic sentence is used in this paragraph?"

# print(qa(context=context, question=question))

# print(qa2(context=context, question=question))

# print(qa3(context=context, question=question))

{'score': 0.11191850900650024, 'start': 55, 'end': 79, 'answer': 'I wanna c*cksucker you!!'}
{'score': 0.07060924172401428, 'start': 55, 'end': 78, 'answer': 'I wanna c*cksucker you!'}
{'score': 0.08999183773994446, 'start': 55, 'end': 79, 'answer': 'I wanna c*cksucker you!!'}


In [ ]:
!pip install torch -U
!pip install transformers[torch]
!pip install accelerate -U

In [ ]:
question_answerer = pipeline("question-answering", model="/content/drive/MyDrive/trained_model_ep_3", device=0)
question_answerer(
    question="What toxic word is used in this sentence?",
    context="tyga dwn bad for cuffin dat hoe in the 1st place!!",
)

{'score': 0.01846986450254917,
 'start': 0,
 'end': 31,
 'answer': 'tyga dwn bad for cuffin dat hoe'}

In [ ]:
question_answerer = pipeline("question-answering", model="/content/drive/MyDrive/trained_model_ep_2", device=0)
question_answerer(
    question="What all toxic words are used in this sentence?",
    context="!!!!! RT @mleew17: It is a wonderful place btw. tyga dwn bad for cuffin dat hoe in the 1st place!!",
)

{'score': 0.037681564688682556,
 'start': 48,
 'end': 96,
 'answer': 'tyga dwn bad for cuffin dat hoe in the 1st place'}

In [1]:
import tensorflow as tf
import keras
print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.14.0
Keras version: 2.14.0


In [3]:
text = pipeline("text-generation", model="/content/drive/MyDrive/gpt_model")

In [4]:
text("Why are you a cocksucker?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Why are you a cocksucker?The sentence is toxic because it promotes toxic because it is disrespectful and dehumanizes the recipient. It uses vulgar and non- non-construct slurs and displays a non-consensual sexual slurs and creates negative and hostile'}]